# <a id='toc1_'></a>[Hacer censo y extraer estables](#toc0_)



**Table of contents**<a id='toc0_'></a>
    
----
- [General](#toc1_1_)    
- [Funciones](#toc1_2_)    
- [_Main_](#toc1_3_)    
----
<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[General](#toc0_)



In [2]:

import pickle
import os
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm as tqdm

general_directory=os.getcwd()+'/Data/'


----
## <a id='toc1_2_'></a>[Funciones](#toc0_)

In [8]:
get_days()

['2022-05-18.dat']

In [4]:
def get_days():
    if not os.path.isdir(general_directory): os.mkdir('Data')
    if not os.path.isdir(general_directory+'Days'): os.mkdir(general_directory+'Days')
    days=[i for i in os.listdir(general_directory+'Days') if i[-3:]=='dat' ]
    days.sort()
    days=[days[i] for i in range(len(days)) if all([(i-j)%7 != 0 for j in [3,4]])]
    return days

In [5]:
def save_dataP(name,dictionary,location=general_directory):

    with open(location+'Saves/'+ name+'.pkl', 'wb') as tf:
        pickle.dump(dictionary,tf)
    

In [6]:
def census(day,cen):
    with open(general_directory+'Days/'+day) as file:
        orow='0'
        for line in file:
            if line!='zones time_end time_start time_start zones\n':
                row=line.replace("'",'').replace("  ",' ').replace('\n','').split(sep=' ')
                if row[0]!=orow[0]:
                    cen[row[0]]=cen.get(row[0],{'home':[],'work':[]})
                    cen[row[0]]['home']=cen[row[0]].get('home')+[row[3]]
                    cen[row[0]]['work']=cen[row[0]].get('work')+[row[4]]
                    orow=row
       
    


In [7]:
def stable(censdic):
    estable={}
    for ID in censdic.keys():
        stable_place=0
        gp=[]
        for place in ['home','work']:
            for i in set(censdic[ID][place]):            
                p=censdic[ID][place].count(i)/N
                if 0.5<=p:
                    stable_place+=1
                    gp.append(i)         
        if stable_place==2:
            estable[int(ID)]=estable.get(int(ID),{'home':[],'work':[]})
            estable[int(ID)]['home']=estable[int(ID)].get('home',gp[0])+[gp[0]]
            estable[int(ID)]['work']=estable[int(ID)].get('work',gp[1])+[gp[1]]
    save_dataP('Estables',estable)
    print('Detectados '+ str(len(estable)) +' IDs con comportamiento estable')


----
## <a id='toc1_3_'></a>[_Main_](#toc0_)

In [ ]:
def main():
    days=get_days()
    N=len(days)
    if N==0: print('No data yet')
    else:
        print('Days in data: '+str(N))
        cen={}
        print('Starting census')
        list(map(census,tqdm(days,desc='Census'),[cen for i in days]))
        #print([str(list(cen.keys())[i])+':'+str(cen[list(cen.keys())[i]]) for i in range(5)])
        print('Defining stables')
        stable(cen)
        print('Census done')

In [ ]:
if __name__=="__main__":
    
    main()